In [24]:
import numpy as np
import importlib

import fastsim

In [25]:
# Load vehicle
veh_name = "2020_Renault_Zoe_E-Tech_Electric_ZE50_snake_case"
veh = fastsim.vehicle.Vehicle.from_file(f"../resources/vehdb/{veh_name}.csv")
veh_rust = veh.to_rust()

In [26]:
# Load WLTC cycle phases
cyc_low = fastsim.cycle.Cycle.from_file("../resources/cycles/wltc_class3_low3.csv")
cyc_med = fastsim.cycle.Cycle.from_file("../resources/cycles/wltc_class3_med3b.csv")
cyc_high = fastsim.cycle.Cycle.from_file("../resources/cycles/wltc_class3_high3b.csv")
cyc_extrahigh = fastsim.cycle.Cycle.from_file("../resources/cycles/wltc_class3_extra_high3.csv")
# Generate combined cycle
cyc_combined_dict = fastsim.cycle.concat(
    [
        cyc_low.get_cyc_dict(),
        cyc_med.get_cyc_dict(),
        cyc_high.get_cyc_dict(),
        cyc_extrahigh.get_cyc_dict(),
    ]
)
cyc = fastsim.cycle.Cycle.from_dict(cyc_combined_dict)
cyc_rust = cyc.to_rust()

In [27]:
# Simulate combined cycle (Python)
sim_python = fastsim.simdrive.SimDrive(cyc, veh)
sim_python.sim_drive()

In [28]:
# Simulate combined cycle (Rust)
sim_rust = fastsim.simdrive.RustSimDrive(cyc_rust, veh_rust)
sim_rust.sim_drive()

In [29]:
i0 = len(cyc_low.time_s)
i1 = i0 + len(cyc_med.time_s)-1
i2 = i1 + len(cyc_high.time_s)-1

low = slice(None, i0)
medium = slice(i0-1, i1)
high = slice(i1-1, i2)
extrahigh = slice(i2-1, None)

In [30]:
meas_Wh_per_km = 177
combined_kWh_per_100km_meas = meas_Wh_per_km / 1000 * 100

kWh_per_100km = sim_rust.electric_kwh_per_mi  * 1000/fastsim.params.M_PER_MI * 100

#print(convert((sim_rust.roadway_chg_kj + sim_rust.ess_dischg_kj) / 3.6e3 / sum(sim_rust.dist_mi)))

print(f"{kWh_per_100km:.3f}")

print(f"{(kWh_per_100km - meas_kWh_per_100km)/meas_kWh_per_100km * 100:+.3f}%")

17.124
-3.255%


In [67]:
def calc_kWh_per_100km(sim, phase):
    # Copy arrays/lists to np.array for calculations
    roadway_chg_kw_out_ach = np.array(sim.roadway_chg_kw_out_ach[phase])
    dt_s = np.array(sim.cyc.dt_s[phase])
    # Calculate energy consumption
    roadway_chg_kj = sum(roadway_chg_kw_out_ach * dt_s)
    ess_dischg_kj = - (sim.soc[phase][-1] - sim.soc[phase][0]) * sim.veh.ess_max_kwh * 3.6e3
    kWh_per_mi = (roadway_chg_kj + ess_dischg_kj) / 3.6e3 / sum(sim.dist_mi[phase])
    kWh_per_100km = kWh_per_mi * 1000/fastsim.params.M_PER_MI * 100
    return kWh_per_100km

def calc_percent_error(simulated, measured):
    return 100 * (simulated - measured) / measured

In [66]:
low_kWh_per_100km_meas = 177 / 1000 * 100
medium_kWh_per_100km_meas = 177 / 1000 * 100
high_kWh_per_100km_meas = 177 / 1000 * 100
extrahigh_kWh_per_100km_meas = 177 / 1000 * 100
combined_kWh_per_100km_meas = 177 / 1000 * 100

In [68]:
low_kWh_per_100km = calc_kWh_per_100km(sim_rust, low)
low_kWh_per_100km_error = calc_percent_error(low_kWh_per_100km, low_kWh_per_100km_meas)

medium_kWh_per_100km = calc_kWh_per_100km(sim_rust, medium)
medium_kWh_per_100km_error = calc_percent_error(medium_kWh_per_100km, medium_kWh_per_100km_meas)

high_kWh_per_100km = calc_kWh_per_100km(sim_rust, high)
high_kWh_per_100km_error = calc_percent_error(high_kWh_per_100km, high_kWh_per_100km_meas)

extrahigh_kWh_per_100km = calc_kWh_per_100km(sim_rust, extrahigh)
extrahigh_kWh_per_100km_error = calc_percent_error(extrahigh_kWh_per_100km, extrahigh_kWh_per_100km_meas)

combined_kWh_per_100km = sim_rust.electric_kwh_per_mi * 1000/fastsim.params.M_PER_MI * 100
combined_kWh_per_100km_error = calc_percent_error(combined_kWh_per_100km, combined_kWh_per_100km_meas)

In [112]:
print("SIMULATED")
print("--------------------------------------")
print(f"LOW:        {low_kWh_per_100km: >6.3f} kWh/100km ({low_kWh_per_100km_error:+.3f}%)")
print(f"MEDIUM:     {medium_kWh_per_100km: >6.3f} kWh/100km ({medium_kWh_per_100km_error:+.3f}%)")
print(f"HIGH:       {high_kWh_per_100km: >6.3f} kWh/100km ({high_kWh_per_100km_error:+.3f}%)")
print(f"EXTRA-HIGH: {extrahigh_kWh_per_100km: >6.3f} kWh/100km ({extrahigh_kWh_per_100km_error:+.3f}%)")
print("--------------------------------------")
print(f"COMBINED:   {combined_kWh_per_100km: >6.3f} kWh/100km ({combined_kWh_per_100km_error: >7.3f}%)")

SIMULATED
--------------------------------------
LOW:        14.443 kWh/100km (-18.403%)
MEDIUM:     13.958 kWh/100km (-21.140%)
HIGH:       15.432 kWh/100km (-12.811%)
EXTRA-HIGH: 21.421 kWh/100km (+21.021%)
--------------------------------------


ValueError: Invalid format specifier